In [144]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
import matplotlib.pyplot as plt

import datetime
import matplotlib.dates as mdates
import matplotlib.cbook as cbook

# read one stock from excel and create dataframe 
def read_clean_stock_from_excel(stock):
    # reading from excel 
    df = pd.read_excel(str(stock) + '.xlsx', sheetname=str(stock))

    # dropping unwanted columns and renaming to english 
    df.drop(['成交股數', '成交金額', '最高價', '開盤價', '最低價', '漲跌價差', '成交筆數'], axis=1, inplace=True)
    df.columns=['date','closing_price']

    # Adding a column of the stock number to the dataframe 
    stock_no = []
    for i in range(len(df['date'])):
        stock_no.append(str(stock))

    df['stock_no'] = stock_no
    
    return df
    

# concatenate 5 stocks into one dataframe
def concatenate_process_5_stocks(stock_list):
    # add dataframes to df_list
    df_list = []
    for stock in stock_list:
        df_list.append(read_clean_stock_from_excel(stock))   
    
    # concat df of different stocks 
    final_df = df_list[0]
    
    for i in range(1, len(df_list)):
        final_df = final_df.append(df_list[i])
    
    # reorganise data pulled by setting date as index with
    # columns of stock_no and their corresponding closing_prices
    clean = final_df.set_index('date')
    #print(clean)
    table = clean.pivot(index=clean.index, columns='stock_no')
    #table.head()  

    # calculate daily and annual returns of the stocks
    returns_daily = table.pct_change() # Percentage change between the current and a prior element.
    returns_annual = returns_daily.mean() * len(table) #trading days

    # get daily and covariance of returns of the stock
    cov_daily = returns_daily.cov()
    cov_annual = cov_daily * len(table)

    # empty lists to store returns, volatility and weights of imaginary portfolios
    port_returns = []
    port_volatility = []
    sharpe_ratio = []
    stock_weights = []

    # set the number of combinations for imaginary portfolios
    num_assets = len(stock_list)
    num_portfolios = 5000


    #set random seed for reproduction's sake
    np.random.seed(101)


    # populate the empty lists with each portfolios returns,risk and weights
    for single_portfolio in range(num_portfolios):
        
        weights = np.random.random(num_assets)
        weights /= np.sum(weights)
        
        returns = np.dot(weights, returns_annual)
        
        volatility = np.sqrt(np.dot(weights.T, np.dot(cov_annual, weights))) #SD
        
        rf_rate = 0.00 # risk free rate of return
        
        sharpe = (returns - rf_rate)/ volatility
        
        sharpe_ratio.append(sharpe)
        port_returns.append(returns)
        port_volatility.append(volatility)
        stock_weights.append(weights)

    # a dictionary for Returns and Risk values of each portfolio
    portfolio = {'Returns': port_returns,
                 'Volatility': port_volatility,
                 'Sharpe Ratio': sharpe_ratio}

    # extend original dictionary to accomodate each ticker and weight in the portfolio
    for counter,stock in enumerate(stock_list):
        portfolio[str(stock)+' Weight'] = [Weight[counter] for Weight in stock_weights]
    
    # make a nice dataframe of the extended dictionary
    df = pd.DataFrame(portfolio)

    # get better labels for desired arrangement of columns
    column_order = ['Returns', 'Volatility', 'Sharpe Ratio'] + [str(stock)+' Weight' for stock in stock_list]

    # reorder dataframe columns
    df = df[column_order]
    
    # find min Volatility & max sharpe values in the dataframe (df)
    min_volatility = df['Volatility'].min()
    max_sharpe = df['Sharpe Ratio'].max()

    # use the min, max values to locate and create the two special portfolios
    # df.loc --> Access a group of rows and columns by label(s) or a boolean array.
    sharpe_portfolio = df.loc[df['Sharpe Ratio'] == max_sharpe] 
    min_variance_port = df.loc[df['Volatility'] == min_volatility]
    
    # print the details of the 2 special portfolios
    print(min_variance_port.T)
    print(sharpe_portfolio.T)
    

    # plot frontier, max sharpe & min Volatility values with a scatterplot
    plt.style.use('seaborn-dark')
    fig, ax = plt.subplots()
    df.plot.scatter(x='Volatility', y='Returns', c='Sharpe Ratio',
                    cmap='RdYlGn', edgecolors='black', figsize=(10, 8), grid=True, ax=ax)
    plt.scatter(x=sharpe_portfolio['Volatility'], y=sharpe_portfolio['Returns'], c='red', marker='D', s=200)
    plt.scatter(x=min_variance_port['Volatility'], y=min_variance_port['Returns'], c='blue', marker='D', s=200 )
    plt.xlabel('Volatility (Std. Deviation)')
    plt.ylabel('Expected Returns')
    plt.title('Optimal Portfolios')
    plt.show() 
    

# The most risk-averse investor would construct the minimum variance portfolio
# which has an expected return of 11.96% with an accompanying expected volatility of 10.27%. 

# Investors seeking the maximum risk-adjusted return would opt for portfolio that with 
# the maximum Sharpe Ratio which has an expected return of 16.33% with expected volatility pegged at 11.68%




stock_list = [1101, 1102, 1216, 1301, 1303]
concatenate_process_5_stocks(stock_list)




                  2409
Returns       0.119639
Volatility    0.102736
Sharpe Ratio  1.164526
1101 Weight   0.028277
1102 Weight   0.013797
1216 Weight   0.227396
1301 Weight   0.178414
1303 Weight   0.552116
                  1715
Returns       0.163333
Volatility    0.116827
Sharpe Ratio  1.398068
1101 Weight   0.349714
1102 Weight   0.220555
1216 Weight   0.001948
1301 Weight   0.100157
1303 Weight   0.327626
